In [ ]:
import torch
import torch.nn as nn

class AnsorFusionModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        # 1. Sketch Encoder (예: BERT)
        self.sketch_embedding = nn.Embedding(vocab_size, hidden_dim) 
        # 실제로는 BERT 모델 전체가 들어갑니다.
        
        # 2. Value Encoder (숫자 -> 벡터)
        # 숫자를 log scale 등으로 전처리 했다고 가정하고 Linear로 받습니다.
        self.value_encoder = nn.Sequential(
            nn.Linear(1, hidden_dim), # 입력이 스칼라 숫자 1개
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, sketch_input_ids, split_values, split_mask_indices):
        """
        sketch_input_ids: (Batch, Seq_Len) - DAG와 스케줄 토큰들
        split_values: (Batch, Max_Split_Len, 1) - 실제 Split 값들 (패딩 포함)
        split_mask_indices: (Batch, Max_Split_Len) - sketch 내에서 [S] 토큰의 위치 인덱스
        """
        
        # --- Step 1: Sketch Encoding ---
        # (Batch, Seq_Len, Hidden)
        sketch_embs = self.sketch_embedding(sketch_input_ids)
        
        # --- Step 2: Value Encoding ---
        # 값들도 배치 처리를 위해 패딩되어 들어옵니다. (Batch, Max_Split, 1)
        # (Batch, Max_Split, Hidden)
        value_embs = self.value_encoder(split_values)
        
        # --- Step 3: Fusion (핵심!) ---
        # sketch_embs의 특정 위치(split_mask_indices)에 value_embs를 더해줍니다.
        
        # scatter를 쓰기 위해 차원을 맞춥니다.
        # split_mask_indices는 (Batch, Max_Split)이므로, 
        # 이를 (Batch, Max_Split, Hidden)으로 확장해야 scatter가 가능합니다.
        batch_size, max_split, hidden = value_embs.shape
        
        # 인덱스 확장: (Batch, Max_Split, 1) -> (Batch, Max_Split, Hidden)
        indices = split_mask_indices.unsqueeze(-1).expand(-1, -1, hidden)
        
        # scatter_add_: sketch_embs의 'indices' 위치에 'value_embs'를 더함
        # sketch_embs는 원본을 유지하면서 복사본을 만드는게 안전합니다.
        fused_embs = sketch_embs.clone()
        fused_embs.scatter_add_(1, indices, value_embs)
        
        return fused_embs

# --- 사용 예시 ---
# 배치 사이즈 1, 시퀀스 길이 10, 히든 4
model = AnsorFusionModel(vocab_size=100, hidden_dim=4)

# DAG+스케줄: [10, 20, 99, 99, 99, 30] (99가 S토큰이라고 가정)
sketch_ids = torch.LongTensor([[10, 20, 99, 99, 99, 30]]) 

# Split 값: [2, 0, 100] (길이 3)
values = torch.FloatTensor([[[2], [0], [100]]]) # (1, 3, 1)

# S토큰의 위치: 2, 3, 4번 인덱스
indices = torch.LongTensor([[2, 3, 4]]) 

# 실행
output = model(sketch_ids, values, indices)

print("결과 텐서 크기:", output.shape) 
# 결과: (1, 6, 4) -> 원래 시퀀스 길이 유지됨. 
# 하지만 2,3,4번 위치는 구조정보+값정보가 합쳐져 있음.